In [4]:
import numpy as np
import pandas as pd
from scipy.special import j1, jn_zeros

# -------------------------
# Grid parameters
# -------------------------
Nr, Ntheta, Nt = 50, 50, 10
r = np.linspace(0, 1, Nr)
theta = np.linspace(0, 2*np.pi, Ntheta)
t = np.linspace(0, 1.0, Nt)
R, Theta, T = np.meshgrid(r, theta, t, indexing='ij')

# -------------------------
# Initial condition
# -------------------------
alpha = jn_zeros(1,1)[0]
U0 = 2*np.cos(Theta[:,:,0]) * j1(alpha * R[:,:,0])

# Flatten branch input
branch_flat = U0.flatten()

# -------------------------
# Placeholder solution (exact solution)
# Here: no advection, u = u0
# Replace with PDE solver output if available
# -------------------------
U_flat = np.tile(branch_flat[:, None], (1, Nt)).flatten()

# -------------------------
# Trunk inputs
# -------------------------
coords = np.stack([R.flatten(), Theta.flatten(), T.flatten()], axis=-1)

# -------------------------
# Combine branch, trunk, and solution into DataFrame
# -------------------------
data_dict = {
    'r': coords[:,0],
    'theta': coords[:,1],
    't': coords[:,2],
    'u_true': U_flat
}

# Add branch features as columns: b0, b1, b2, ...
for i, val in enumerate(branch_flat):
    data_dict[f'b{i}'] = val

df = pd.DataFrame(data_dict)

# -------------------------
# Save CSV
# -------------------------
df.to_csv('deeponet_2d_advection_data.csv', index=False)
print("CSV file generated: deeponet_2d_advection_data.csv")


CSV file generated: deeponet_2d_advection_data.csv


In [5]:
import numpy as np
import pandas as pd
from scipy.special import j1, jn_zeros

# -------------------------
# Grid parameters
# -------------------------
Nr, Ntheta, Nt = 20, 5, 10
r = np.linspace(0, 1, Nr)
theta = np.linspace(0, 2*np.pi, Ntheta)
t = np.linspace(0, 1.0, Nt)
R, Theta, T = np.meshgrid(r, theta, t, indexing='ij')

# -------------------------
# Initial condition
# -------------------------
alpha = jn_zeros(1,1)[0]
U0 = 2*np.cos(Theta[:,:,0]) * j1(alpha * R[:,:,0])

# Flatten branch input
branch_flat = U0.flatten()

# -------------------------
# Placeholder solution (exact solution)
# Here: no advection, u = u0
# Replace with PDE solver output if available
# -------------------------
U_flat = np.tile(branch_flat[:, None], (1, Nt)).flatten()

# -------------------------
# Trunk inputs
# -------------------------
coords = np.stack([R.flatten(), Theta.flatten(), T.flatten()], axis=-1)

# -------------------------
# Combine branch, trunk, and solution into DataFrame
# -------------------------
data_dict = {
    'r': coords[:,0],
    'theta': coords[:,1],
    't': coords[:,2],
    'u_true': U_flat
}

# Add branch features as columns: b0, b1, b2, ...
for i, val in enumerate(branch_flat):
    data_dict[f'b{i}'] = val

df = pd.DataFrame(data_dict)

# -------------------------
# Save CSV
# -------------------------
df.to_csv('deeponet_2d_advection_data1.csv', index=False)
print("CSV file generated: deeponet_2d_advection_data1.csv")

CSV file generated: deeponet_2d_advection_data1.csv


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

# -------------------------
# 1. Load CSV Data
# -------------------------
df = pd.read_csv("deeponet_2d_advection_data1.csv")

# Trunk inputs (coords)
trunk_input = df[['r', 'theta', 't']].values.astype(np.float32)

# Branch inputs (initial condition vector, all b0,b1,... columns)
branch_cols = [c for c in df.columns if c.startswith("b")]
branch_input = df[branch_cols].values.astype(np.float32)

# Target output
u_true = df[['u_true']].values.astype(np.float32)

# -------------------------
# 2. Dataset + Dataloader
# -------------------------
class DeepONetDataset(Dataset):
    def __init__(self, branch, trunk, target):
        self.branch = torch.tensor(branch)
        self.trunk = torch.tensor(trunk)
        self.target = torch.tensor(target)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return self.branch[idx], self.trunk[idx], self.target[idx]

dataset = DeepONetDataset(branch_input, trunk_input, u_true)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# -------------------------
# 3. Define DeepONet
# -------------------------
class DeepONet(nn.Module):
    def __init__(self, branch_in, trunk_in, hidden=64, out_dim=1):
        super(DeepONet, self).__init__()
        # Branch net
        self.branch_net = nn.Sequential(
            nn.Linear(branch_in, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden)
        )
        # Trunk net
        self.trunk_net = nn.Sequential(
            nn.Linear(trunk_in, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden)
        )
        # Output projection
        self.output_layer = nn.Linear(hidden, out_dim)

    def forward(self, branch_input, trunk_input):
        b = self.branch_net(branch_input)
        t = self.trunk_net(trunk_input)
        # element-wise product then projection
        x = b * t
        return self.output_layer(x)

branch_in = branch_input.shape[1]
trunk_in = trunk_input.shape[1]
model = DeepONet(branch_in, trunk_in)

# -------------------------
# 4. Training
# -------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for b, t, u in dataloader:
        optimizer.zero_grad()
        u_pred = model(b, t)
        loss = criterion(u_pred, u)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss = {total_loss/len(dataloader):.6f}")

# -------------------------
# 5. Save trained model
# -------------------------
torch.save(model.state_dict(), "deeponet_advection_trained.pth")
print("✅ Training finished, model saved.")


Epoch 1/20, Loss = 0.357955
Epoch 2/20, Loss = 0.328358
Epoch 3/20, Loss = 0.251713
Epoch 4/20, Loss = 0.179758
Epoch 5/20, Loss = 0.160360
Epoch 6/20, Loss = 0.146520
Epoch 7/20, Loss = 0.136806
Epoch 8/20, Loss = 0.128210
Epoch 9/20, Loss = 0.122509
Epoch 10/20, Loss = 0.107000
Epoch 11/20, Loss = 0.108257
Epoch 12/20, Loss = 0.094770
Epoch 13/20, Loss = 0.094923
Epoch 14/20, Loss = 0.090577
Epoch 15/20, Loss = 0.084302
Epoch 16/20, Loss = 0.080398
Epoch 17/20, Loss = 0.083064
Epoch 18/20, Loss = 0.080205
Epoch 19/20, Loss = 0.074869
Epoch 20/20, Loss = 0.070666
✅ Training finished, model saved.


In [1]:
import os
print(os.getcwd())          # current folder
print(os.listdir())         # list of files


C:\Users\mpstme.student\Advection
['.ipynb_checkpoints', 'Adv-PCA.ipynb', 'Adv.EGI.ipynb', 'Adv.Eqn Dataset1.ipynb', 'Adv.polar.ipynb', 'Adv1-cartesian form.ipynb', 'Adv2-modes-cartesian form.ipynb', 'advection_dataset.csv', 'Advec_Deep.ipynb', 'ADvEqn FBNN.ipynb', 'Adv_Deep.ipynb', 'deeponet_2d_advection.pth', 'deeponet_2d_advection_data.csv', 'deeponet_2d_advection_data1.csv', 'deeponet_advection_trained.pth', 'Exact solution for Advection FB.ipynb', 'fbnn_model.pth', 'FBNO-Advection.ipynb', 'polar_advection_solution.csv', 'u_data.npy', 'u_recon.npy']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.special import j1, jn_zeros

# -------------------------
# 1. Load CSV data (r, theta, t, u)
# -------------------------
df = pd.read_csv("your_data.csv")   # must contain columns r, theta, t, u

r = df["r"].values.astype(np.float32)
theta = df["theta"].values.astype(np.float32)
t = df["t"].values.astype(np.float32)
u_exact = df["u"].values.astype(np.float32)

X = np.stack([r, theta, t], axis=1)
y = u_exact.reshape(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# -------------------------
# 2. Prepare Branch Input (initial condition samples)
# -------------------------
# Discretize r,theta grid for branch input
Nr, Ntheta = 20, 20
r_grid = np.linspace(0, 1, Nr)
theta_grid = np.linspace(0, 2*np.pi, Ntheta, endpoint=False)
R, TH = np.meshgrid(r_grid, theta_grid, indexing="ij")

# First zero of J1
alpha11 = jn_zeros(1, 1)[0]

# Initial condition u0(r,theta)
U0 = 2 * np.cos(TH) * j1(alpha11 * R)

# Flatten as branch input vector
branch_input_vector = U0.flatten().astype(np.float32)
branch_input_vector = torch.tensor(branch_input_vector, dtype=torch.float32).unsqueeze(0)  # shape (1, Nr*Ntheta)

# Repeat for all samples
branch_input_train = branch_input_vector.repeat(X_train.shape[0], 1)
branch_input_test = branch_input_vector.repeat(X_test.shape[0], 1)

# -------------------------
# 3. Define DeepONet
# -------------------------
class DeepONet(nn.Module):
    def __init__(self, branch_dim):
        super(DeepONet, self).__init__()
        # Branch net (initial condition)
        self.branch = nn.Sequential(
            nn.Linear(branch_dim, 128), nn.ReLU(),
            nn.Linear(128, 128), nn.ReLU()
        )
        # Trunk net (query point (r,θ,t))
        self.trunk = nn.Sequential(
            nn.Linear(3, 128), nn.ReLU(),
            nn.Linear(128, 128), nn.ReLU()
        )
        self.fc = nn.Linear(128, 1)

    def forward(self, branch_input, trunk_input):
        b = self.branch(branch_input)
        t = self.trunk(trunk_input)
        return self.fc(b * t)  # elementwise product

# Initialize model
branch_dim = Nr * Ntheta
model = DeepONet(branch_dim)

# -------------------------
# 4. Training
# -------------------------
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(branch_input_train, X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Train Loss = {loss.item():.4e}")

# -------------------------
# 5. Predictions and Relative L2 error
# -------------------------
model.eval()
with torch.no_grad():
    y_pred_test = model(branch_input_test, X_test).numpy().squeeze()
    y_true_test = y_test.numpy().squeeze()

# Relative L2 Error
numerator = np.linalg.norm(y_true_test - y_pred_test, 2)
denominator = np.linalg.norm(y_true_test, 2)
rel_L2_error = numerator / denominator
print(f"\nRelative L2 Error = {rel_L2_error:.4e}")

# -------------------------
# 6. Plots
# -------------------------

# Exact vs Predicted
plt.figure(figsize=(6,6))
plt.scatter(y_true_test, y_pred_test, alpha=0.7)
plt.plot([y_true_test.min(), y_true_test.max()],
         [y_true_test.min(), y_true_test.max()], 'r--')
plt.xlabel("Exact u")
plt.ylabel("Predicted u")
plt.title("Exact vs Predicted (DeepONet)")
plt.grid(True)
plt.show()

# Error distribution
errors = y_true_test - y_pred_test
plt.figure(figsize=(6,4))
plt.hist(errors, bins=40, alpha=0.7, color='blue')
plt.xlabel("Prediction Error (u_exact - u_pred)")
plt.ylabel("Frequency")
plt.title("Error Distribution")
plt.grid(True)
plt.show()
